In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import dask.dataframe as dd

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

## Download the data 

In [2]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "../data/" # This notebook should be ran mannually

response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]

for file in files:
    if file["name"] == "data.zip":
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

In [3]:
output_directory = "../data/" # This notebook should be ran mannually

## Combine the data

In [5]:
%%time
%%memit

combined_data = dd.from_pandas(pd.DataFrame({'time': [], "lat_min": [], "lat_max": [], "lon_min": [], "lon_max": [], "rain (mm/day)": [], "model": []}), npartitions=1)

for filename in os.listdir(output_directory):
    if filename[-4: ] == ".csv":
        model = filename.partition('_daily_rainfall')[0]
        ddf = dd.read_csv(output_directory + filename, assume_missing=True)
        if len(ddf.columns) == 2:
            ddf['lat_min'] = None
            ddf['lat_max'] = None
            ddf['lon_min'] = None
            ddf['lon_max'] = None
            ddf = ddf[['time', 'lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']]
        ddf["model"] = model
        combined_data = dd.concat([combined_data, ddf], axis=0)

peak memory: 173.79 MiB, increment: 8.83 MiB
CPU times: user 631 ms, sys: 62.1 ms, total: 693 ms
Wall time: 817 ms


In [6]:
%%time
%%memit

combined_data.to_csv(output_directory + "combined_data.csv")

peak memory: 3246.38 MiB, increment: 3074.11 MiB
CPU times: user 13min 40s, sys: 1min 6s, total: 14min 47s
Wall time: 12min 48s


## EDA in python

In [7]:
combined_data = dd.read_csv(output_directory + "combined_data.csv/*")

In [17]:
combined_data = combined_data.drop(['Unnamed: 0'], axis = 1)

## EDA in R